In [1]:
import psycopg2
import pandas as pd
import hidden

In [3]:
sql_string = hidden.psycopg2(hidden.secrets())
print('PostgreSQL connection data taken from hidden.py')

# Make the connection and cursor
conn = psycopg2.connect(sql_string, connect_timeout=3)

PostgreSQL connection data taken from hidden.py


In [4]:
pd.read_sql_query('''SELECT * FROM pg_catalog.pg_tables WHERE schemaname = 'public'
                  ''', conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,candlestick_15m,master_pg_user,None,True,False,True,False
1,public,pairs,master_pg_user,None,True,False,True,False


In [13]:
pd.read_sql_query('''SELECT * FROM candlestick_15m LIMIT 10
                  ''', conn)

,pair_id,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,created_at,updated_at
0,0,2021-03-16 03:15:00,0.032240,0.032269,0.032179,0.032223,2896.399,2021-03-16 03:29:59.999,93.310285,2796,1508.270,48.587970,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
1,0,2021-03-16 03:30:00,0.032222,0.032253,0.032147,0.032200,2829.272,2021-03-16 03:44:59.999,91.107113,2651,1385.149,44.600536,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
2,0,2021-03-16 03:45:00,0.032195,0.032296,0.032183,0.032214,3344.664,2021-03-16 03:59:59.999,107.796917,2615,1690.294,54.477293,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
3,0,2021-03-16 04:00:00,0.032216,0.032244,0.032099,0.032100,3093.701,2021-03-16 04:14:59.999,99.541708,2524,1573.360,50.621399,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
4,0,2021-03-16 04:15:00,0.032100,0.032154,0.032032,0.032105,4054.045,2021-03-16 04:29:59.999,130.112279,3536,2048.317,65.739699,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
5,0,2021-03-16 04:30:00,0.032106,0.032233,0.032053,0.032191,3164.468,2021-03-16 04:44:59.999,101.837363,2354,1119.178,35.999865,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
6,0,2021-03-16 04:45:00,0.032190,0.032206,0.032150,0.032173,2531.480,2021-03-16 04:59:59.999,81.463569,1905,1047.721,33.715457,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
7,0,2021-03-16 05:00:00,0.032172,0.032347,0.032167,0.032281,3023.726,2021-03-16 05:14:59.999,97.552813,3132,1553.120,50.107438,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
8,0,2021-03-16 05:15:00,0.032281,0.032380,0.032246,0.032367,2016.597,2021-03-16 05:29:59.999,65.185199,2153,1093.856,35.358488,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00
9,0,2021-03-16 05:30:00,0.032367,0.032477,0.032361,0.032439,4305.843,2021-03-16 05:44:59.999,139.569104,3893,1782.137,57.782789,2021-06-30 14:28:24.981992+00:00,2021-07-01 10:57:33.052002+00:00


In [31]:
pd.read_sql_query('''
SELECT DATE_TRUNC('day', open_time) as date,
       COALESCE(SUM(CASE WHEN open < 33995.87 THEN taker_buy_base_asset_volume END),0) as "low_volume",
       COALESCE(SUM(CASE WHEN open >= 33995.87 THEN taker_buy_base_asset_volume END),0) as "high_volume"
 FROM candlestick_15m
WHERE open_time > (CURRENT_DATE - INTERVAL '30 days')
GROUP BY date
ORDER BY date
''', conn)

,date,low_volume,high_volume
0,2021-06-12,4.677749e+05,42042.323349
1,2021-06-13,4.356054e+05,45675.273616
2,2021-06-14,4.228127e+05,53645.908744
3,2021-06-15,3.685934e+05,39428.829726
4,2021-06-16,3.462298e+05,42137.242251
5,2021-06-17,3.308297e+05,37936.812673
6,2021-06-18,4.329944e+05,44824.436567
7,2021-06-19,2.986415e+05,33788.293345
8,2021-06-20,4.464613e+05,35853.829513
9,2021-06-21,9.682801e+05,13670.958443


In [6]:
sql = """
SELECT count(*)
    --cs.*
FROM candlestick_15m cs
INNER JOIN pairs p
    ON p.id = cs.pair_id
WHERE p.symbol LIKE 'ETHBTC'
-- ORDER BY cs.open_time desc
-- LIMIT 1
"""
df = pd.read_sql_query(sql, conn)
df

,count
0,138614


In [7]:
sql = """
SELECT cs.*
FROM candlestick_15m cs
INNER JOIN pairs p
    ON p.id = cs.pair_id
WHERE p.symbol LIKE 'ETHBTC'
ORDER BY cs.open_time desc
"""
df = pd.read_sql_query(sql, conn)
df

,pair_id,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,created_at,updated_at
0,0,2021-07-02 04:00:00,0.062028,0.062098,0.061952,0.062070,1109.910,NaT,68.858576,1528,499.305,30.978440,2021-07-02 04:00:21.599261+00:00,2021-07-02 04:10:21.669405+00:00
1,0,2021-07-02 03:45:00,0.062334,0.062367,0.061986,0.062038,2196.710,2021-07-02 03:59:59.000,136.456879,3911,1281.412,79.591470,2021-07-02 03:45:21.613905+00:00,2021-07-02 04:00:21.599261+00:00
2,0,2021-07-02 03:30:00,0.062785,0.062785,0.062267,0.062326,2269.186,2021-07-02 03:44:59.000,141.858378,4303,1050.212,65.659002,2021-07-02 03:30:21.555569+00:00,2021-07-02 03:45:21.613905+00:00
3,0,2021-07-02 03:15:00,0.063087,0.063201,0.062751,0.062783,1650.940,2021-07-02 03:29:59.000,104.022572,2608,746.156,47.023885,2021-07-02 03:15:33.157623+00:00,2021-07-02 03:30:21.555569+00:00
4,0,2021-07-02 03:00:00,0.062944,0.063175,0.062896,0.063087,1210.511,2021-07-02 03:14:59.000,76.283867,1829,715.191,45.064299,2021-07-02 03:00:12.340782+00:00,2021-07-02 03:15:33.157623+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138609,0,2017-07-14 05:00:00,0.085874,0.086205,0.084608,0.084680,16.520,2017-07-14 05:14:59.999,1.404228,15,1.176,0.100914,2021-06-30 14:24:31.900306+00:00,2021-07-01 10:57:33.052002+00:00
138610,0,2017-07-14 04:45:00,0.086314,0.086380,0.086309,0.086347,42.818,2017-07-14 04:59:59.999,3.697253,61,7.948,0.686295,2021-06-30 14:24:31.900306+00:00,2021-07-01 10:57:33.052002+00:00
138611,0,2017-07-14 04:30:00,0.085811,0.086380,0.085811,0.086314,53.769,2017-07-14 04:44:59.999,4.631740,41,21.225,1.829643,2021-06-30 14:24:31.900306+00:00,2021-07-01 10:57:33.052002+00:00
138612,0,2017-07-14 04:15:00,0.085289,0.086000,0.085128,0.085811,61.042,2017-07-14 04:29:59.999,5.217079,33,2.738,0.235027,2021-06-30 14:24:31.900306+00:00,2021-07-01 10:57:33.052002+00:00


In [8]:
import datetime
timestamp = datetime.datetime.now().strftime('%y%m%d%H%M')
timestamp

'2107021313'

In [9]:
df.to_pickle(f'../data/candlestick_15m_ETHBTC_{timestamp}.pkl')